In [116]:
import pandas as pd
import json
import warnings
import os
import numpy as np

warnings.filterwarnings('ignore', category = UserWarning, module = 'openpyxl')
warnings.filterwarnings('ignore', category = RuntimeWarning)

with open('LIMPIADOR_TOTAL.json', 'r', encoding = 'utf-8') as f:
    DICCIONARIO_LIMPIADOR_NOMENCLATURAS = json.load(f)

    LIMPIADOR_EVE = DICCIONARIO_LIMPIADOR_NOMENCLATURAS['NOMENCLATURA EVE']
    LIMPIADOR_REDUNDANTES = DICCIONARIO_LIMPIADOR_NOMENCLATURAS['REDUNDANTES']

    LIMPIADOR_GLOBAL = LIMPIADOR_EVE | LIMPIADOR_REDUNDANTES

help(warnings.filterwarnings)

Help on function filterwarnings in module warnings:

filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)
    Insert an entry into the list of warnings filters (at the front).
    
    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters



In [147]:
class Estadisticas:
    def __init__(self):
        pass

    def obtener_estadisticas_todos_los_años(self, output):
        nombres_sensibilidades_ordenadas = sorted([excel for excel in os.listdir() if 'Fibrosis' in excel])
        estadisticas = {'MICROORG MAS FRECUENTES': {},
                        'RESISTENCIAS GLOBALES': {},
                        'PERFILES ESPECIFICOS': {}}

        for nombre_archivo in nombres_sensibilidades_ordenadas:
            print(f'Leyendo {nombre_archivo}')
            microorg_mas_frecuentes, resist_globales, perfiles = self.obtener_estadistica_por_archivo(nombre_archivo)

            estadisticas['MICROORG MAS FRECUENTES'][nombre_archivo] = (microorg_mas_frecuentes)
            estadisticas['RESISTENCIAS GLOBALES'][nombre_archivo] = (resist_globales)
            estadisticas['PERFILES ESPECIFICOS'][nombre_archivo] = (perfiles)
        
        with pd.ExcelWriter('ESTADISTICAS_TOTALES.xlsx') as writer:
            for tipo_estadistica, estadisticas_por_año in estadisticas.items():
                i = 0
                fila_inicio = 0
                for año, resultado_estadistica in estadisticas_por_año.items():                    
                    df_estadistica = pd.DataFrame.from_dict(dict(resultado_estadistica), orient = 'index')
                    df_estadistica.index.name = año

                    if tipo_estadistica != 'PERFILES ESPECIFICOS':
                        df_estadistica.to_excel(writer, sheet_name = tipo_estadistica, startcol = i * 3)
                    
                    else:
                        
                        df_estadistica.to_excel(writer, sheet_name = tipo_estadistica, startrow = fila_inicio)
                        fila_inicio += df_estadistica.shape[0] + 3



                    i += 1
                
                if tipo_estadistica != 'PERFILES ESPECIFICOS':
                    df_acumulada = pd.DataFrame.from_dict(dict(estadisticas[tipo_estadistica]), orient = 'index')
                    df_acumulada.to_excel(writer, sheet_name = tipo_estadistica, startcol = i * 3)
                



    def obtener_estadistica_por_archivo(self, nombre_archivo):
        df = pd.read_excel(nombre_archivo, header = 1).dropna(how = 'all')
        #df = df.fillna('-')
        df_limpia = df.copy()
        df_limpia['MICROORGANISMO'] = df_limpia['MICROORGANISMO'].map(lambda x: LIMPIADOR_GLOBAL[x] if (LIMPIADOR_GLOBAL[x] != None) else x)
        mask_candidas_patogenos = df_limpia['MICROORGANISMO'].apply(lambda x: True if (('Candida' in x) or \
                                                                   (x == 'Streptococcus alfa hemolitico') or \
                                                                   (x == 'Staphylococcus pseudintermedius') or \
                                                                   (x == 'Staphylococcus coagulasa (-)') or \
                                                                   (x == 'Streptococcus alfa hemolitico') or \
                                                                   (x == 'Staphylococcus epidermidis') or \
                                                                   (x == 'Staphylococcus warneri') or \
                                                                   (x == 'Staphylococcus pasteuri') or \
                                                                   (x == 'Streptococcus viridans') or \
                                                                   (x == 'Staphylococcus haemolyticus') or \
                                                                   (x == 'No hubo desarrollo bacteriano a las 24 horas de incubación.')) else False)
        
        df_limpia = df_limpia[~mask_candidas_patogenos]

        estadisticas_fq = self.calcular_estadisticas(df_limpia)

        return estadisticas_fq

    def calcular_estadisticas(self, df_fq):
        frecuencias_microorganismos = self.calcular_microorganismos_mas_frecuentes(df_fq)
        sensibilidades_globales = self.calcular_sensibilidades_globales(df_fq)
        sensibilidades_microorganismos = self.calcular_sensibilidades_por_microorganismo(df_fq)

        estadisticas_totales = [frecuencias_microorganismos, sensibilidades_globales, sensibilidades_microorganismos]

        return estadisticas_totales

    
    def calcular_microorganismos_mas_frecuentes(self, df_fq):
        return df_fq['MICROORGANISMO'].value_counts()
    
    def calcular_sensibilidades_globales(self, df_fq):
        sensibilidades = {}
        for farmaco in df_fq.loc[:, 'AK': 'VAN'].columns:
            filas_que_si_se_les_midio = df_fq[farmaco]
            recuento_valores = filas_que_si_se_les_midio.value_counts()
        
            total = recuento_valores.sum()

            try:
                sensibles = recuento_valores['S']
            
            except KeyError:
                sensibles = 0
            
            try:
                resistentes = recuento_valores['R']
            
            except KeyError:
                resistentes = 0
            
            try:
                intermedio = recuento_valores['I']
            except KeyError:
                intermedio = 0
            
            porcentaje_sensibles = round((sensibles / total) * 100, 1)
            # porcentaje_resistentes = round((resistentes / total) * 100, 1)
            # porcentaje_intermedios = round((intermedio / total) * 100, 1)

            # resistencias[farmaco] = (porcentaje_sensibles, porcentaje_resistentes, porcentaje_intermedios)
            sensibilidades[farmaco] = porcentaje_sensibles
            
        return sensibilidades
    
    def calcular_sensibilidades_por_microorganismo(self, df_fq):
        sensibilidades = {}

        for microorganismo in df_fq['MICROORGANISMO'].unique():
            df_micro = df_fq[df_fq['MICROORGANISMO'] ==  microorganismo]
            sensibilidades[microorganismo] = self.calcular_sensibilidades_globales(df_micro)
        
        return sensibilidades


In [148]:
estadisticas = Estadisticas()
resultados = estadisticas.obtener_estadisticas_todos_los_años('ESTADISTICAS_FQ_3.xlsx')

Leyendo Fibrosis 2016.xlsx
Leyendo Fibrosis 2017.xlsx
Leyendo Fibrosis 2018.xlsx
Leyendo Fibrosis 2019.xlsx
Leyendo Fibrosis 2020.xlsx
Leyendo Fibrosis 2021.xlsx
Leyendo Fibrosis 2022.xlsx
